In [23]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F

import numpy as np


class FeedForwardNN(nn.Module):
    '''
        A standard two-hidden-layer Feed Forward Neural Network
    '''
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int, custom_init_behavior: bool = False, output_bias: Tensor = None) -> None:
        '''
            Initialize the network and set up the layers.

            Parameters:
                in_dim - input dimensions as an int
                hidden_dim - hidden layer dimensions as an int
                out_dim - output dimensions as an int
                custom_init_behavior - boolean to reduce weights of last layer by 100 and add custom bias if given
                output_bias - replace the last layer's bias with this value if custom_init_behavior is true

            Return:
                None
        '''
        super(FeedForwardNN, self).__init__()

        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, out_dim)

        if custom_init_behavior:
            self.fc4.weight.data = self.fc4.weight.data / 100
            if output_bias is not None:
                self.fc4.bias.data = output_bias

    def forward(self, obs: Tensor) -> Tensor:
        '''
            Runs a forward pass on the neural network

            Parameters:
                obs - observation to pass as input (either Tensor or converts to Tensor)

            Return:
                output - the output of the forward pass
        '''
        if not isinstance(obs, Tensor):
            obs = torch.tensor(obs, dtype=torch.float)

        activation1 = F.tanh(self.fc1(obs))
        activation2 = F.tanh(self.fc2(activation1))
        activation3 = F.tanh(self.fc3(activation2))
        output = self.fc4(activation3)

        return output

In [22]:
model = FeedForwardNN(22, 64, 22, custom_init_behavior=True)

print("model.fc4.bias:", model.fc4.bias.data)

outputs = []
with torch.no_grad():
    for _ in range(10000):
        outputs.append(model(torch.rand((22))))

outputs = torch.tensor(np.array(outputs))
print("Means:",outputs.mean(dim=0))
print("Std:", outputs.std(dim=0))

model.fc4.bias: tensor([-0.0898,  0.0250,  0.0651, -0.0432, -0.0453,  0.0802,  0.0067,  0.1089,
        -0.0059, -0.0660, -0.0907,  0.0558,  0.0027, -0.0742, -0.0887, -0.1058,
         0.0910,  0.0751, -0.0180,  0.1168,  0.1106, -0.0666])
Means: tensor([-0.0897,  0.0248,  0.0639, -0.0442, -0.0444,  0.0791,  0.0074,  0.1071,
        -0.0050, -0.0654, -0.0905,  0.0568,  0.0023, -0.0735, -0.0874, -0.1059,
         0.0914,  0.0750, -0.0177,  0.1167,  0.1095, -0.0670])
Std: tensor([0.0003, 0.0002, 0.0004, 0.0002, 0.0004, 0.0003, 0.0002, 0.0005, 0.0003,
        0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0002,
        0.0002, 0.0003, 0.0003, 0.0002])
